# Importing the libraries

In [1]:
import tensorflow as tf
import os
from tqdm import tqdm
import cv2
import numpy as np

# Hyperparameter

In [2]:
MODEL_PATH = 'models/segmentation_model_3/model.tflite'
SEG_TEST_DATASET_DIR = 'datasets/segmentation_datasets/iris_needle_seg_dataset_mixed/'

IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE_SEG = 8

interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

# Importing the dataset

In [3]:
# Load Test Dataset OG Image and Mask Image Path
test_image_path = []
test_mask_path = []
for root, dirs, files in os.walk(SEG_TEST_DATASET_DIR + 'test_original'):
    for file in files:
        path_og = os.path.join(root,file)
        path_mask = os.path.join(SEG_TEST_DATASET_DIR + 'test_mask/', file)
        test_image_path.append(path_og)
        test_mask_path.append(path_mask)

# Verify Length and Consistency OG and Mask Image
print(len(test_image_path))
print(test_image_path[1])
print(len(test_mask_path))
print(test_mask_path[1])

# Load and Grayscale Mask Image
test_masks = []
for path in tqdm(test_mask_path):
    mask = cv2.imread(path)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    test_masks.append(mask)

# Load and Grayscale OG Image
test_images = []
for path in tqdm(test_image_path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test_images.append(image)

# Clear Outlier
for mask in test_masks:
    mask[mask < 127] = 0
    mask[mask >= 127] = 255

129
datasets/segmentation_datasets/iris_needle_seg_dataset_mixed/test_original\SID0016_jpg.rf.fe55049434804bb6bf8f761e303b4a61.jpg
129
datasets/segmentation_datasets/iris_needle_seg_dataset_mixed/test_mask/SID0016_jpg.rf.fe55049434804bb6bf8f761e303b4a61.jpg


100%|██████████| 129/129 [00:01<00:00, 92.81it/s]


# IoU Scoring

In [20]:
def calculate_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

iou_scores = []

_, height, width, _ = interpreter.get_input_details()[0]['shape']

# Iterate over the test data
for i, (img, true_mask) in enumerate(zip(test_images, test_masks)):
    # Load Image from Path and Resize
    img = cv2.resize(img, (width, height)).astype(np.float32)
    true_mask = cv2.resize(true_mask, (width, height)).astype(np.float32)
    img = img / 255.0
    
    # Add Batch Dimension
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)

    # Set Input and Invoke Segmentation Model
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], img)
    interpreter.invoke()

    # Get Output
    pred_mask = np.squeeze(interpreter.get_tensor(interpreter.get_output_details()[0]['index']))
    
    # Threshold the Output Segmentation
    pred_mask = np.where(pred_mask > 0, 1, 0).astype(np.uint8)

    # Calculate the IoU and append to the list
    iou_score = calculate_iou(true_mask, pred_mask)
    iou_scores.append(iou_score)
    
    # Save the first prediction and mask
    if i == 0:
        cv2.imwrite('first_prediction.png', pred_mask*255)
        cv2.imwrite('first_mask.png', true_mask*255)
        
# Calculate the average IoU score
average_iou = np.mean(iou_scores)
print(iou_scores)
print("Average IoU score on the test set: ", average_iou)

[0.54315185546875, 0.3802490234375, 0.45806884765625, 0.4366455078125, 0.45306396484375, 0.42572021484375, 0.48681640625, 0.48553466796875, 0.51446533203125, 0.44366455078125, 0.5701904296875, 0.4481201171875, 0.56842041015625, 0.6480712890625, 0.5328369140625, 0.52093505859375, 0.5257568359375, 0.59478759765625, 0.67431640625, 0.61956787109375, 0.555908203125, 0.57421875, 0.62445068359375, 0.60113525390625, 0.5693359375, 0.61224365234375, 0.47576904296875, 0.58551025390625, 0.53363037109375, 0.45587158203125, 0.66009521484375, 0.52703857421875, 0.51092529296875, 0.4649658203125, 0.62109375, 0.60205078125, 0.64752197265625, 0.70068359375, 0.68316650390625, 0.53912353515625, 0.49139404296875, 0.473876953125, 0.54266357421875, 0.65374755859375, 0.53057861328125, 0.5496826171875, 0.501220703125, 0.46246337890625, 0.39483642578125, 0.61029052734375, 0.4072265625, 0.48779296875, 0.64410400390625, 0.4769287109375, 0.44976806640625, 0.56585693359375, 0.59765625, 0.47357177734375, 0.5388183593